In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import pickle

import numpy as np
import xarray as xr

import synthia as syn
import pyvinecopulib as pv

from collections import defaultdict

import sys
sys.path.append('../src')

from util import load_ds_inputs, to_stacked_array, compute_layer_longwave_downwelling

In [ ]:
copula_type = str(os.environ.get('copula_type', 'gaussian'))
has_targets = int(os.environ.get('has_targets', 1))
idx_iter = int(os.environ.get('idx_iter', 2))

num_threads = int(os.environ.get('num_threads', 2))

is_normalised = 0
data_fraction = 1.0

print(copula_type, has_targets, num_threads, idx_iter)

In [ ]:
fname = f"stats-copula_type={copula_type}-has_targets={has_targets}-idx_iter={idx_iter}.pkl"
outdir = Path.cwd().parent / 'results' / 'stats'
outdir.mkdir(parents=True, exist_ok=True)
fpath = outdir / fname

if fpath.exists():
    raise RuntimeError('This case is already present. Skipping...')

In [ ]:
PROJ_PATH = Path.cwd().parent
ds_true_in = load_ds_inputs(PROJ_PATH)

if has_targets:
    column_gas_optical_depth = 1.7
    flux_dn_hl_train = compute_layer_longwave_downwelling(ds_true_in, column_gas_optical_depth)
    ds_true = xr.merge([ds_true_in, flux_dn_hl_train])
else:
    ds_true = ds_true_in

ds_true, _ = to_stacked_array(ds_true)
display(ds_true)

In [ ]:
fname = f"copula_type={copula_type}-has_targets={has_targets}-is_normalised={is_normalised}-data_fraction={data_fraction}.pkl"
outdir = PROJ_PATH / 'results' / 'fitting'

with open(outdir / fname, 'rb') as f:
    generator = pickle.load(f)

In [ ]:
kws = dict(seed=idx_iter)
if copula_type != 'gaussian':
    kws['num_threads'] = num_threads

ds_synth = generator.generate(ds_true.shape[0], **kws)
ds_synth, _ = to_stacked_array(ds_synth)
ds_synth

In [ ]:
np.random.seed(idx_iter)
weights = np.random.rand(ds_true.shape[1], 1)

proj_true = np.dot(ds_true, weights)
proj_synth = np.dot(ds_synth, weights)

In [ ]:
stats = {}
stats_types = [{
        'name': 'Mean',
        'fn': lambda arr: np.mean(arr)
    },
    {
        'name': 'Median',
        'fn': lambda arr: np.median(arr)
    },
    {
        'name': 'Variance',
        'fn': lambda arr: np.var(arr)
    },
    {
        'name': 'Standard deviation',
        'fn': lambda arr: np.std(arr)
    }, 
    {
        'name': '0.1-quantile',
        'fn': lambda arr: np.quantile(arr, 0.1)
    },
    {
        'name': '0.5-quantile',
        'fn': lambda arr: np.quantile(arr, 0.5)
    },
    {
        'name': '0.9-quantile',
        'fn': lambda arr: np.quantile(arr, 0.9)
    }]

for stats_type in stats_types:
    proj_true_stat = stats_type['fn'](proj_true)
    proj_pred_stat = stats_type['fn'](proj_synth)

    stats[stats_type['name']] = {
        'true' : proj_true_stat,
        'pred' :  proj_pred_stat
    }

In [ ]:
pickle.dump(stats, open(fpath, 'wb'))

# Test
with open(fpath, 'rb') as f:
    stats = pickle.load(f)
stats